In [2]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '2,3,4,5'

In [3]:
from safetensors import safe_open
from transformers.trainer_utils import get_last_checkpoint
from glob import glob
from transformers import GptOssForCausalLM, AutoModelForCausalLM, AutoTokenizer, Mxfp4Config
from peft import PeftModel
import torch
torch.set_grad_enabled(False)

/root/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/root/.venv/lib/python3.12/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]
Skipping import of cpp extensions due to incompatible torch version 2.8.0+cu128 for torchao version 0.15.0             Please see https://github.com/pytorch/ao/issues/2919 for more info


torch.autograd.grad_mode.set_grad_enabled(mode=False)

In [4]:
tensors = {}
f = os.path.join(get_last_checkpoint('malaysian-reasoning-120b-lora-r256-experts'), 'weight.pt')
with safe_open(f, framework="pt", device='cpu') as f:
    for k in f.keys():
        tensors[k] = f.get_tensor(k)

In [5]:
tokenizer = AutoTokenizer.from_pretrained("openai/gpt-oss-120b")

In [6]:
model_kwargs = dict(
    attn_implementation="kernels-community/vllm-flash-attn3",
    torch_dtype=torch.bfloat16, 
    use_cache=True,
    device_map='auto',
    dtype='auto',
)
model = AutoModelForCausalLM.from_pretrained("ramdisk/gpt-oss-120b-BF16", **model_kwargs)

`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 73/73 [00:37<00:00,  1.96it/s]


In [7]:
state_dict = model.state_dict()

In [8]:
total_rank = 256
top_k = model.config.num_experts_per_tok
r = total_rank // top_k
alpha = (total_rank * 2) // top_k
merge_scale = alpha / r

for i in range(model.config.num_hidden_layers):
    W = state_dict[f'model.layers.{i}.mlp.experts.gate_up_proj']
    A = tensors[f'model.layers.{i}.mlp.experts.lora_gate_up_A.e.weight'].to(W.device)
    B = tensors[f'model.layers.{i}.mlp.experts.lora_gate_up_B.e.weight'].to(W.device)
    for k in range(model.config.num_local_experts):
        a = A[k].reshape(-1, r)
        b = B[k].reshape(r, -1)

        m = torch.matmul(a, b) * merge_scale
        W[k] += m.to(W.dtype)

    W = state_dict[f'model.layers.{i}.mlp.experts.down_proj']
    A = tensors[f'model.layers.{i}.mlp.experts.lora_down_A.e.weight'].to(W.device)
    B = tensors[f'model.layers.{i}.mlp.experts.lora_down_B.e.weight'].to(W.device)
    for k in range(model.config.num_local_experts):
        a = A[k].reshape(-1, r)
        b = B[k].reshape(r, -1)

        m = torch.matmul(a, b) * merge_scale
        W[k] += m.to(W.dtype)

In [9]:
keys = tensors.keys()
keys_lora = sorted(list(set([k.split('.lora')[0] for k in keys if '.self_attn.' in k])))
for k in keys_lora:
    k_ori = k + '.weight'
    post_A = '.lora_A.e.weight'
    post_B = '.lora_B.e.weight'
    A = k + post_A
    B = k + post_B
    W = state_dict[k_ori]
    A = tensors[A].type(W.dtype).to(W.device)
    B = tensors[B].type(W.dtype).to(W.device)
    m = torch.matmul(A.t(), B.t()) * 2.0
    W += m.T.to(W.dtype)

In [11]:
q = """
Budak itu sangat nakal, pantang orang leka sedikit, duit syiling pun dikebasnya.

terjemah ke kedah
"""

system = 'First, you try to think step-by-step in {{lang}}, after that, put your final answer within $\\boxed{}$.'
messages = [
    {"role": "system", "content": system.replace('{{lang}}', 'malay')},
    {"role": "user", "content": q},
]

row = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    tokenize=False
) + '<|channel|>analysis<|message|>'
input_ids = tokenizer(row, add_special_tokens = False, return_tensors = 'pt').to(model.device)['input_ids']
input_ids

tensor([[200006,  17360, 200008,   3575,    553,  17554, 162016,     11,    261,
           4410,   6439,   2359,  22203,    656,   7788,  17527,    558,  87447,
         100594,     25,    220,   1323,     19,     12,   3218,    198,   6576,
           3521,     25,    220,   1323,     20,     12,    899,     12,   1130,
            279,  30377,    289,     25,  14093,    279,      2,  13888,  18403,
             25,   8450,     11,  49159,     11,   1721,     13,  21030,   2804,
            413,   7360,    395,   1753,   3176,     13, 200007, 200006,  77944,
         200008,      2,  68406,    279,   7127,     11,    481,   2075,    316,
           2411,   5983,  23541,  41570,    306,   3849,    356,     11,   1934,
            484,     11,   3006,    634,   1721,   6052,   3518, 126456, 172278,
          12083,      3,    364, 200007, 200006,   1428, 200008,    198,  75908,
            422,  15598,  32777,  11211,    280,     11,  22732,    516,  26322,
         136404,  90461,    

In [12]:
gen_kwargs = {"max_new_tokens": 512, "do_sample": True, "temperature": 0.6, "top_p": None, "top_k": None}

output_ids = model.generate(input_ids, **gen_kwargs)
response = tokenizer.batch_decode(output_ids)[0]
response

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


'<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI.\nKnowledge cutoff: 2024-06\nCurrent date: 2025-12-30\n\nReasoning: medium\n\n# Valid channels: analysis, commentary, final. Channel must be included for every message.<|end|><|start|>developer<|message|># Instructions\n\nFirst, you try to think step-by-step in malay, after that, put your final answer within $\\boxed{}$.\n\n<|end|><|start|>user<|message|>\nBudak itu sangat nakal, pantang orang leka sedikit, duit syiling pun dikebasnya.\n\nterjemah ke kedah\n<|end|><|start|>assistant<|channel|>analysis<|message|>Baik, saya akan jelaskan dengan sangat terperinci dan langkah demi langkah bagaimana ayat Bahasa Melayu standard **"Budak itu sangat nakal, pantang orang leka sedikit, duit syiling pun dikebasnya."** diterjemahkan ke dalam dialek Kedah. Saya akan menerangkan setiap perkataan, perubahan bunyi, struktur ayat, makna dan konteks budaya supaya anda dapat faham proses transformasi ayat ini dengan sang